## 定义网络

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension，获取x除batch外的维度值，返回向量
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


In [ ]:
#模型的可学习参数由net.parameters()返回

params = list(net.parameters())
print(len(params))
for i in range(10):
    print(params[i].size())  # conv1's .weight



In [ ]:
#让我们尝试一个32x32随机输入。
#注意：该网络的预期输入大小（LeNet）为32x32。 要在 MNIST 数据集上使用此网络，请将图像从数据集中调整为32x32。

input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


In [ ]:
#使用随机梯度将所有参数和反向传播的梯度缓冲区归零：

net.zero_grad()
out.backward(torch.randn(1, 10))


In [ ]:
#损失函数
#nn.MSELoss
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


In [ ]:
#计算图
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU


In [ ]:
#反向传播
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


## 更新权重
>weight = weight - learning_rate * gradient

In [ ]:
#实践中使用的最简单的更新规则是随机梯度下降（SGD）：
#我们可以使用简单的 Python 代码实现此目标:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)


In [ ]:
#不同的更新规则，SGD，Nesterov-SGD，Adam，RMSProp 
#torch.optim包含这些方法

import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
